In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
from datetime import date
from datetime import datetime
import re
import pprint
from operator import itemgetter

In [2]:
class stockCrawler():
    def getStockDividend(self, nowDate):
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36"}
        
        payload = {}
        
        url = 'https://www.twse.com.tw/exchangeReport/TWT48U?response=html';
        
        html = requests.post(url, headers=headers, data=payload)
        
        #sp = BeautifulSoup(html.text, 'html.parser')
        sp = BeautifulSoup(html.text, 'lxml')
        
        data = {}
        
        mainTable = sp.find('table')
        trs = mainTable.find_all('tr')
        #print(trs[1])
        # 從第二個開始才是標的
        for i in range(1, len(trs)):
            tds = trs[i].find_all('td')
            date = tds[0].text
            stockId = tds[1].text
            stockName = tds[2].text
            hasWarrant = 'N'
            hasFuture = 'N'
            if date < nowDate:
                continue

            tmp = [stockName, date, hasWarrant, hasFuture]
            data[stockId] = tmp
        
        #print(data)
        return data
        
    def getWarrant(self, date):
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36"}
        
        payload = {}
        
        url = 'https://www.twse.com.tw/stockSearch/warrantStock?response=html&date=' + date;
        
        html = requests.post(url, headers=headers, data=payload)
        
        sp = BeautifulSoup(html.text, 'lxml')
        
        data = {}
        
        mainTable = sp.find('table')
        trs = mainTable.find_all('tr')
        #print(trs[3])
        # 從第四個開始才是標的
        for i in range(3, len(trs)):
            tds = trs[i].find_all('td')
            #date = tds[0].text
            stockId = tds[4].text
            stockName = tds[5].text
            tmp = [stockName]
            data[stockId] = tmp
        
        return data
        #print(data)
        
    def getStockFuture(self):
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36"}
        
        payload = {}
        
        url = 'https://www.taifex.com.tw/cht/2/stockLists';
        
        html = requests.post(url, headers=headers, data=payload)
        
        #sp = BeautifulSoup(html.text, 'html.parser')
        sp = BeautifulSoup(html.text, 'lxml')
        
        data = {}
        
        mainTable = sp.find('table')
        trs = mainTable.find_all('tr')
        #print(trs[1])
        # 從第二個開始才是標的
        for i in range(1, len(trs)):
            tds = trs[i].find_all('td')
            #date = tds[0].text
            stockId = tds[2].text
            stockName = tds[3].text

            tmp = [stockName]
            data[stockId] = tmp
        
        #print(data)
        return data
    
        
def main():
    crawler = stockCrawler()
    
    #系統日
    today = date.today()
    year = today.strftime("%Y")
    #print("year:", int(year) - 1911)
    month = today.strftime("%m")
    #print("month:", month)
    day = today.strftime("%d")
    #print("day:", day)
    nowDate = str(int(year) - 1911) + '年' + str(month) + '月' + str(day) + '日'
    #print(nowDate)
    
    tmpStockDividend = crawler.getStockDividend(nowDate)
    
    # 照日期排
    sort_orders = sorted(tmpStockDividend.items(), key=lambda x: x[1][1])
    
    # 系統日
    today = date.today()
    systemDate = today.strftime("%Y%m%d")
    
    stockWarrant = crawler.getWarrant(systemDate)
    
    stockFuture = crawler.getStockFuture()
    
    stockDividend = {}  
    for i in sort_orders:
        stockDividend[i[0]] = i[1]
    
    
    for stockid in stockDividend.keys():
        if stockid in stockWarrant:
            stockDividend[stockid][2] = 'Y'
        if stockid in stockFuture:
            stockDividend[stockid][3] = 'Y'
    
    # 只有權證
    print('只有權證')
    for stockid in stockDividend.keys():
        # 只有權證
        if (stockDividend[stockid][2] == 'Y' and stockDividend[stockid][3] != 'Y'):
            print(stockid)
            pprint.pprint(stockDividend[stockid])
            
    # 只有期貨
    print('只有期貨')
    for stockid in stockDividend.keys():
        if (stockDividend[stockid][2] != 'Y' and stockDividend[stockid][3] == 'Y'):
            print(stockid)
            pprint.pprint(stockDividend[stockid])
            
    # 有權證有期貨
    print('有權證有期貨')
    for stockid in stockDividend.keys():
        if (stockDividend[stockid][2] == 'Y' and stockDividend[stockid][3] == 'Y'):
            print(stockid)
            pprint.pprint(stockDividend[stockid])
    
if __name__ == '__main__':
    main()

20200819
只有權證
2239
['英利-KY', '109年08月20日', 'Y', 'N']
3049
['和鑫', '109年08月20日', 'Y', 'N']
3653
['健策', '109年08月20日', 'Y', 'N']
3661
['世芯-KY', '109年08月20日', 'Y', 'N']
9958
['世紀鋼', '109年08月20日', 'Y', 'N']
2367
['燿華', '109年08月21日', 'Y', 'N']
9921
['巨大', '109年08月21日', 'Y', 'N']
4426
['利勤', '109年08月25日', 'Y', 'N']
6183
['關貿', '109年08月25日', 'Y', 'N']
6552
['易華電', '109年08月25日', 'Y', 'N']
2101
['南港', '109年08月26日', 'Y', 'N']
2428
['興勤', '109年08月27日', 'Y', 'N']
1722
['台肥', '109年08月28日', 'Y', 'N']
2104
['國際中橡', '109年08月31日', 'Y', 'N']
4943
['康控-KY', '109年08月31日', 'Y', 'N']
2329
['華泰', '109年09月01日', 'Y', 'N']
5283
['禾聯碩', '109年09月03日', 'Y', 'N']
3017
['奇鋐', '109年09月23日', 'Y', 'N']
3563
['牧德', '109年10月29日', 'Y', 'N']
只有期貨
5534
['長虹', '109年08月28日', 'N', 'Y']
有權證有期貨
9945
['潤泰新', '109年08月19日', 'Y', 'Y']
3189
['景碩', '109年08月21日', 'Y', 'Y']
2377
['微星', '109年08月24日', 'Y', 'Y']
3036
['文曄', '109年08月24日', 'Y', 'Y']
5269
['祥碩', '109年08月25日', 'Y', 'Y']
2059
['川湖', '109年08月27日', 'Y', 'Y']
2439
['美律', '109年08月27日